In [2]:
import cv2
import pytesseract
import pandas as pd
from datetime import datetime
from ultralytics import YOLO
import csv

pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"  # Raw string for Windows path
model = YOLO("Yolo_Plate.pt")

cap = cv2.VideoCapture("mycarplate.mp4")

processed_numbers = set()
output_file = "car_plate_data.csv"


with open(output_file, "a+") as file:
    if file.tell() == 0:  
        file.write("NumberPlate,Date,Time\n")

frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    frame_count += 1
    if frame_count % 3 != 0:  
        continue
    frame = cv2.resize(frame, (1080, 500))
    results = model.predict(frame)

    for box in results[0].boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        conf = box.conf.item()

        plate_roi = frame[y1:y2, x1:x2]
        gray = cv2.cvtColor(plate_roi, cv2.COLOR_BGR2GRAY)
        gray = cv2.bilateralFilter(gray, 10, 20, 20)
        
        text = pytesseract.image_to_string(
            gray,
            config='--psm 8 --oem 3 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'
        ).strip().replace(" ", "").replace("(", "").replace(")", "").replace(",", "")
        
        if text and text not in processed_numbers:
            processed_numbers.add(text)
            current_time = datetime.now().strftime("%Y-%m-%d,%H:%M:%S")
            
            with open(output_file, "a") as file:
                file.write(f"{text},{current_time}\n")
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, text, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
            cv2.imshow("License Plate", plate_roi)
    
    cv2.imshow("Frame", frame)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


0: 320x640 (no detections), 191.9ms
Speed: 3.8ms preprocess, 191.9ms inference, 0.6ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 plate, 207.6ms
Speed: 3.5ms preprocess, 207.6ms inference, 1.2ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 plate, 182.8ms
Speed: 2.3ms preprocess, 182.8ms inference, 1.2ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 plate, 145.8ms
Speed: 2.4ms preprocess, 145.8ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 plate, 148.0ms
Speed: 2.0ms preprocess, 148.0ms inference, 1.1ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 plate, 134.0ms
Speed: 1.7ms preprocess, 134.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 plate, 142.6ms
Speed: 1.9ms preprocess, 142.6ms inference, 0.9ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 plate, 154.8ms
Speed: 3.4ms preprocess, 154.8ms inference, 0.9ms postprocess per image at